In [1]:
import re
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

### Reading block dates in from ipblocks 

In [6]:
#ipblocks = pd.read_csv("/home/ec2-user/SageMaker/bucket/wiki_trust/ipblocks_fulldump_new.csv")
#del ipblocks['Unnamed: 0']
ipblocks = pd.read_csv("/home/ec2-user/SageMaker/bucket/wiki_trust/ipblocks_fulldump_20190223.txt",sep='\t')
ipblocks.tail()

,ipb_address,date
1166567,Sad.tbhs,20190217
1166568,NaN,20190217
1166569,NaN,20190217
1166570,Ieatpussy4747,20190217
1166571,NaN,20190217


In [7]:
ipblocks = ipblocks[['ipb_address','date']]
ipblocks.tail()

,ipb_address,date
1166567,Sad.tbhs,20190217
1166568,NaN,20190217
1166569,NaN,20190217
1166570,Ieatpussy4747,20190217
1166571,NaN,20190217


#### Removing the null rows

In [8]:
ipblocks.isnull().sum()

ipb_address    274
date             0
dtype: int64

In [9]:
ipblocks.shape

(1166572, 2)

In [10]:
ipblocks = ipblocks[ipblocks['ipb_address'].isnull()==False]

In [11]:
ipblocks.isnull().sum()

ipb_address    0
date           0
dtype: int64

In [12]:
ipblocks.shape

(1166298, 2)

### Developing for single user (Skip to Iterative code)

In [23]:
ipblocks[ipblocks['ipb_address']=='Sad.tbhs']

,ipb_address,date
1166567,Sad.tbhs,20190217


In [24]:
buser = 'Sad.tbhs'

In [25]:
bdate = '20190217'

In [26]:
new_url_1 = 'https://en.wikipedia.org/w/index.php?title=Special:Contributions&offset='+bdate+'235959&limit=20&contribs=user&target='+buser+'&namespace=3&tagfilter=&start=&end='
new_url_1

'https://en.wikipedia.org/w/index.php?title=Special:Contributions&offset=20190217235959&limit=20&contribs=user&target=Sad.tbhs&namespace=3&tagfilter=&start=&end='

In [27]:
response1 = requests.get(new_url_1)
html1 = response1.content
soup1= BeautifulSoup(html1,'html.parser')

In [28]:
print(soup1.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   User contributions for Sad.tbhs - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"Special","wgCanonicalSpecialPageName":"Contributions","wgNamespaceNumber":-1,"wgPageName":"Special:Contributions","wgTitle":"Contributions","wgCurRevisionId":0,"wgRevisionId":0,"wgArticleId":0,"wgIsArticle":false,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September",

In [29]:
type(soup1)

bs4.BeautifulSoup

#### Shortening to only needed lines

In [30]:
list(soup1.find('div',id='mw-content-text').children)[4].get_text()

AttributeError: 'NavigableString' object has no attribute 'get_text'

In [31]:
for i in list(soup1.find('div',id='mw-content-text')):
    try:
        if i.get_text() == 'No changes were found matching these criteria.\n':
            print('yes')
        else:
            print('no')
    except:
        continue        

no
no
no
no
no
no
no


In [32]:
if list(soup1.find('div',id='mw-content-text').children)[1].get_text() == 'No changes were found matching these criteria.\n':
    print('yes')

In [33]:
soup1 = soup1.find('ul',class_='mw-contributions-list')
soup1

<ul class="mw-contributions-list">
<li class="mw-contributions-current mw-tag-mobile_edit mw-tag-mobile_web_edit" data-mw-revid="883692999"><a class="mw-changeslist-date" href="/w/index.php?title=User_talk:Sad.tbhs&amp;oldid=883692999" title="User talk:Sad.tbhs">00:13, 17 February 2019</a>
<span class="mw-changeslist-links"><span><a class="mw-changeslist-diff" href="/w/index.php?title=User_talk:Sad.tbhs&amp;diff=prev&amp;oldid=883692999" title="User talk:Sad.tbhs">diff</a></span> <span><a class="mw-changeslist-history" href="/w/index.php?title=User_talk:Sad.tbhs&amp;action=history" title="User talk:Sad.tbhs">hist</a></span></span> <span class="mw-changeslist-separator"></span> <span class="mw-plusminus-pos mw-diff-bytes" dir="ltr" title="5,167 bytes after change of this size">+142</span>‎ <span class="mw-changeslist-separator"></span>
<a class="mw-contributions-title" href="/wiki/User_talk:Sad.tbhs" title="User talk:Sad.tbhs">User talk:Sad.tbhs</a>
‎ <span class="comment comment--witho

In [34]:
type(soup1)

bs4.element.Tag

In [35]:
lines = soup1.findAll('li')

In [36]:
lines

[<li class="mw-contributions-current mw-tag-mobile_edit mw-tag-mobile_web_edit" data-mw-revid="883692999"><a class="mw-changeslist-date" href="/w/index.php?title=User_talk:Sad.tbhs&amp;oldid=883692999" title="User talk:Sad.tbhs">00:13, 17 February 2019</a>
 <span class="mw-changeslist-links"><span><a class="mw-changeslist-diff" href="/w/index.php?title=User_talk:Sad.tbhs&amp;diff=prev&amp;oldid=883692999" title="User talk:Sad.tbhs">diff</a></span> <span><a class="mw-changeslist-history" href="/w/index.php?title=User_talk:Sad.tbhs&amp;action=history" title="User talk:Sad.tbhs">hist</a></span></span> <span class="mw-changeslist-separator"></span> <span class="mw-plusminus-pos mw-diff-bytes" dir="ltr" title="5,167 bytes after change of this size">+142</span>‎ <span class="mw-changeslist-separator"></span>
 <a class="mw-contributions-title" href="/wiki/User_talk:Sad.tbhs" title="User talk:Sad.tbhs">User talk:Sad.tbhs</a>
 ‎ <span class="comment comment--without-parentheses"><span dir="auto

In [37]:
type(lines)

bs4.element.ResultSet

In [38]:
len(lines)

3

In [158]:
type(lines[0])

bs4.element.Tag

In [159]:
list(lines[0].children)

[<a class="mw-redirect mw-changeslist-date" href="/w/index.php?title=User_talk:Kumioko&amp;oldid=541956284" title="User talk:Kumioko">22:37, 3 March 2013</a>,
 '\n',
 <span class="mw-changeslist-links"><span><a class="mw-redirect mw-changeslist-diff" href="/w/index.php?title=User_talk:Kumioko&amp;diff=prev&amp;oldid=541956284" title="User talk:Kumioko">diff</a></span> <span><a class="mw-redirect mw-changeslist-history" href="/w/index.php?title=User_talk:Kumioko&amp;action=history" title="User talk:Kumioko">hist</a></span></span>,
 ' ',
 <span class="mw-changeslist-separator"></span>,
 ' ',
 <strong class="mw-plusminus-pos mw-diff-bytes" dir="ltr" title="11,574 bytes after change of this size">+1,052</strong>,
 '\u200e ',
 <span class="mw-changeslist-separator"></span>,
 '\n',
 <a class="mw-redirect mw-contributions-title" href="/w/index.php?title=User_talk:Kumioko&amp;redirect=no" title="User talk:Kumioko">User talk:Kumioko</a>,
 '\n\u200e ',
 <span class="comment comment--without-pare

In [163]:
date = list(lines[0].children)[0].get_text()
date

'22:37, 3 March 2013'

In [164]:
char_change = list(lines[0].children)[6].get_text()
char_change

'+1,052'

In [165]:
li = list(lines[0].children)[2]
li

<span class="mw-changeslist-links"><span><a class="mw-redirect mw-changeslist-diff" href="/w/index.php?title=User_talk:Kumioko&amp;diff=prev&amp;oldid=541956284" title="User talk:Kumioko">diff</a></span> <span><a class="mw-redirect mw-changeslist-history" href="/w/index.php?title=User_talk:Kumioko&amp;action=history" title="User talk:Kumioko">hist</a></span></span>

In [166]:
type(li)

bs4.element.Tag

In [167]:
list(li.children)

[<span><a class="mw-redirect mw-changeslist-diff" href="/w/index.php?title=User_talk:Kumioko&amp;diff=prev&amp;oldid=541956284" title="User talk:Kumioko">diff</a></span>,
 ' ',
 <span><a class="mw-redirect mw-changeslist-history" href="/w/index.php?title=User_talk:Kumioko&amp;action=history" title="User talk:Kumioko">hist</a></span>]

In [168]:
diff = list(li.children)[0]
diff

<span><a class="mw-redirect mw-changeslist-diff" href="/w/index.php?title=User_talk:Kumioko&amp;diff=prev&amp;oldid=541956284" title="User talk:Kumioko">diff</a></span>

In [169]:
type(diff)

bs4.element.Tag

In [170]:
type(list(diff.children)[0])

bs4.element.Tag

In [172]:
list(diff.children)[0]

<a class="mw-redirect mw-changeslist-diff" href="/w/index.php?title=User_talk:Kumioko&amp;diff=prev&amp;oldid=541956284" title="User talk:Kumioko">diff</a>

In [173]:
link = list(diff.children)[0].get('href')
link

'/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541956284'

#### Now trying this iteratively to get all 20 links for namespace3

In [245]:
new_url_1 = 'https://en.wikipedia.org/w/index.php?title=Special:Contributions&offset='+bdate+'235959&limit=20&contribs=user&target='+buser+'&namespace=3&tagfilter=&start=&end='
new_url_1

'https://en.wikipedia.org/w/index.php?title=Special:Contributions&offset=20140601235959&limit=20&contribs=user&target=Kumioko&namespace=3&tagfilter=&start=&end='

In [246]:
response1 = requests.get(new_url_1)
html1 = response1.content
soup1= BeautifulSoup(html1,'html.parser')

In [247]:
soup1 = soup1.find('ul',class_='mw-contributions-list')
lines = soup1.findAll('li')

In [249]:
dates = []
char_changes = []
links = []

In [250]:
for line in lines:
    date = list(line.children)[0].get_text()
    char_change = list(line.children)[6].get_text()
    li = list(line.children)[2]
    diff = list(li.children)[0]
    if(list(diff.children)[0]=='diff'):
        continue
    link = list(diff.children)[0].get('href')
    dates.append(date)
    char_changes.append(char_change)
    links.append(link)

In [251]:
dates

['22:37, 3 March 2013',
 '22:06, 3 March 2013',
 '18:03, 3 March 2013',
 '17:33, 3 March 2013',
 '17:33, 3 March 2013',
 '17:31, 3 March 2013',
 '17:22, 3 March 2013',
 '16:22, 3 March 2013',
 '15:05, 3 March 2013',
 '14:40, 3 March 2013',
 '14:19, 3 March 2013',
 '03:37, 3 March 2013',
 '03:32, 3 March 2013',
 '19:03, 2 March 2013',
 '19:02, 2 March 2013',
 '19:36, 27 February 2013',
 '02:54, 27 February 2013',
 '02:51, 27 February 2013',
 '02:49, 27 February 2013',
 '02:25, 27 February 2013']

In [252]:
char_changes

['+1,052',
 '+93',
 '+621',
 '+10',
 '+146',
 '+786',
 '+447',
 '+450',
 '+1,059',
 '+676',
 '+280',
 '+476',
 '+555',
 '+194',
 '+538',
 '+282',
 '+540',
 '+286',
 '+280',
 '+22']

In [253]:
links

['/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541956284',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541952363',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541919285',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541915177',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541915064',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541914835',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541913771',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541906603',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541897958',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541895265',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541893207',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541835160',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541834622',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541769064',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=5417688

#### Similarly adding the lines from namespace1

In [254]:
new_url_2 = 'https://en.wikipedia.org/w/index.php?title=Special:Contributions&offset='+bdate+'235959&limit=20&contribs=user&target='+buser+'&namespace=1&tagfilter=&start=&end='
new_url_2

'https://en.wikipedia.org/w/index.php?title=Special:Contributions&offset=20140601235959&limit=20&contribs=user&target=Kumioko&namespace=1&tagfilter=&start=&end='

In [257]:
response2 = requests.get(new_url_2)
html2 = response2.content
soup2= BeautifulSoup(html2,'html.parser')

In [258]:
print(soup2.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   User contributions for Kumioko - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"Special","wgCanonicalSpecialPageName":"Contributions","wgNamespaceNumber":-1,"wgPageName":"Special:Contributions","wgTitle":"Contributions","wgCurRevisionId":0,"wgRevisionId":0,"wgArticleId":0,"wgIsArticle":false,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","

In [259]:
soup2 = soup2.find('ul',class_='mw-contributions-list')
lines = soup2.findAll('li')
lines

[<li data-mw-revid="527069154"><a class="mw-changeslist-date" href="/w/index.php?title=Talk:Boolean_algebra&amp;oldid=527069154" title="Talk:Boolean algebra">20:06, 8 December 2012</a>
 <span class="mw-changeslist-links"><span><a class="mw-changeslist-diff" href="/w/index.php?title=Talk:Boolean_algebra&amp;diff=prev&amp;oldid=527069154" title="Talk:Boolean algebra">diff</a></span> <span><a class="mw-changeslist-history" href="/w/index.php?title=Talk:Boolean_algebra&amp;action=history" title="Talk:Boolean algebra">hist</a></span></span> <span class="mw-changeslist-separator"></span> <strong class="mw-plusminus-pos mw-diff-bytes" dir="ltr" title="52,879 bytes after change of this size">+617</strong>‎ <span class="mw-changeslist-separator"></span>
 <a class="mw-contributions-title" href="/wiki/Talk:Boolean_algebra" title="Talk:Boolean algebra">Talk:Boolean algebra</a>
 ‎ <span class="comment comment--without-parentheses"><span dir="auto"><span class="autocomment"><a href="/wiki/Talk:Boole

In [236]:
#list(lines[1].children)[0].get_text()

'02:52, 16 October 2012'

In [237]:
#list(lines[1].children)[6].get_text()

'+86'

In [238]:
#list(lines[1].children)[2]

<span class="mw-changeslist-links"><span>diff</span> <span><a class="mw-changeslist-history" href="/w/index.php?title=Talk:Wustl_student_life&amp;action=history" title="Talk:Wustl student life">hist</a></span></span>

In [239]:
#li = list(lines[1].children)[2]
#li

<span class="mw-changeslist-links"><span>diff</span> <span><a class="mw-changeslist-history" href="/w/index.php?title=Talk:Wustl_student_life&amp;action=history" title="Talk:Wustl student life">hist</a></span></span>

In [240]:
#list(li.children)

[<span>diff</span>,
 ' ',
 <span><a class="mw-changeslist-history" href="/w/index.php?title=Talk:Wustl_student_life&amp;action=history" title="Talk:Wustl student life">hist</a></span>]

In [241]:
#diff = list(li.children)[0]
#diff

<span>diff</span>

In [244]:
#list(diff.children)[0]

'diff'

In [243]:
#list(diff.children)[0].get('href')

AttributeError: 'NavigableString' object has no attribute 'get'

In [235]:
#link = list(diff.children)[0].get('href')
#link

'/w/index.php?title=Talk:Boolean_algebra&diff=prev&oldid=527069154'

In [260]:
for line in lines:
    date = list(line.children)[0].get_text()
    char_change = list(line.children)[6].get_text()
    li = list(line.children)[2]
    diff = list(li.children)[0]
    if(list(diff.children)[0]=='diff'):
        continue
    link = list(diff.children)[0].get('href')
    dates.append(date)
    char_changes.append(char_change)
    links.append(link)

In [262]:
dates

['22:37, 3 March 2013',
 '22:06, 3 March 2013',
 '18:03, 3 March 2013',
 '17:33, 3 March 2013',
 '17:33, 3 March 2013',
 '17:31, 3 March 2013',
 '17:22, 3 March 2013',
 '16:22, 3 March 2013',
 '15:05, 3 March 2013',
 '14:40, 3 March 2013',
 '14:19, 3 March 2013',
 '03:37, 3 March 2013',
 '03:32, 3 March 2013',
 '19:03, 2 March 2013',
 '19:02, 2 March 2013',
 '19:36, 27 February 2013',
 '02:54, 27 February 2013',
 '02:51, 27 February 2013',
 '02:49, 27 February 2013',
 '02:25, 27 February 2013',
 '20:06, 8 December 2012']

In [263]:
char_changes

['+1,052',
 '+93',
 '+621',
 '+10',
 '+146',
 '+786',
 '+447',
 '+450',
 '+1,059',
 '+676',
 '+280',
 '+476',
 '+555',
 '+194',
 '+538',
 '+282',
 '+540',
 '+286',
 '+280',
 '+22',
 '+617']

In [261]:
links

['/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541956284',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541952363',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541919285',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541915177',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541915064',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541914835',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541913771',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541906603',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541897958',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541895265',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541893207',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541835160',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541834622',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=541769064',
 '/w/index.php?title=User_talk:Kumioko&diff=prev&oldid=5417688

In [265]:
user = pd.DataFrame({
        "user": buser, 
        "dates": dates, 
        "char_changes": char_changes, 
        "links":links
    })
user

,char_changes,dates,links,user
0,"+1,052","22:37, 3 March 2013",/w/index.php?title=User_talk:Kumioko&diff=prev...,Kumioko
1,+93,"22:06, 3 March 2013",/w/index.php?title=User_talk:Kumioko&diff=prev...,Kumioko
2,+621,"18:03, 3 March 2013",/w/index.php?title=User_talk:Kumioko&diff=prev...,Kumioko
3,+10,"17:33, 3 March 2013",/w/index.php?title=User_talk:Kumioko&diff=prev...,Kumioko
4,+146,"17:33, 3 March 2013",/w/index.php?title=User_talk:Kumioko&diff=prev...,Kumioko
5,+786,"17:31, 3 March 2013",/w/index.php?title=User_talk:Kumioko&diff=prev...,Kumioko
6,+447,"17:22, 3 March 2013",/w/index.php?title=User_talk:Kumioko&diff=prev...,Kumioko
7,+450,"16:22, 3 March 2013",/w/index.php?title=User_talk:Kumioko&diff=prev...,Kumioko
8,"+1,059","15:05, 3 March 2013",/w/index.php?title=User_talk:Kumioko&diff=prev...,Kumioko
9,+676,"14:40, 3 March 2013",/w/index.php?title=User_talk:Kumioko&diff=prev...,Kumioko


### Stage 1: Iterative through list of users to get revision links(read from pandas)

In [2]:
#blocked = pd.read_csv("/home/ec2-user/SageMaker/bucket/wiki_trust/revisions_data/csvs_stored/blocked_users_1718.csv")
blocked = pd.read_csv("/home/ec2-user/SageMaker/bucket/wiki_trust/revisions_data/csvs_stored/ipb_remaining3.csv")
blocked.head(20)

,rev_user_text
0,Pplswater
1,Renamed user 4b5S1pYYpXJgazt
2,Magic BD
3,MnetIsALyingScumbag
4,InhaleTheMemes
5,Philisgay69
6,Idiom thebeast331
7,Yesindeed010
8,Krishna 88
9,Stagarchaeology


In [3]:
blocked.shape

(33998, 1)

In [4]:
ipb = blocked.iloc[20000:,]
#ipb = blocked
ipb

,rev_user_text
20000,The Legit Cyka Blyat
20001,Lolkekgod123xd
20002,Mayurikkeni
20003,Poiuyrewqlkjhgfddsa
20004,Translationpk
20005,Pxbx23092
20006,LULtraviolet
20007,Mrpoopybutthole2
20008,George Papadopoulos 69
20009,Moroccoco


In [13]:
ipb = pd.merge(ipb,ipblocks,how="left",left_on=["rev_user_text"],right_on=["ipb_address"])
ipb.head()

,rev_user_text,ipb_address,date
0,The Legit Cyka Blyat,The Legit Cyka Blyat,20180305
1,Lolkekgod123xd,Lolkekgod123xd,20170515
2,Mayurikkeni,Mayurikkeni,20171112
3,Poiuyrewqlkjhgfddsa,Poiuyrewqlkjhgfddsa,20170520
4,Translationpk,Translationpk,20170323


In [14]:
ipb.isnull().sum()

rev_user_text    0
ipb_address      0
date             0
dtype: int64

In [15]:
del ipb['ipb_address']
ipb.head()

,rev_user_text,date
0,The Legit Cyka Blyat,20180305
1,Lolkekgod123xd,20170515
2,Mayurikkeni,20171112
3,Poiuyrewqlkjhgfddsa,20170520
4,Translationpk,20170323


In [16]:
buser = []
dates = []
char_changes = []
links = []
exc_count1 = 0
exc_count2 = 0
exc_count3 = 0
exc_count4 = 0
user_less_than3 = []
namespace = []
user_all = pd.DataFrame({"user": buser,"dates": dates,"char_changes": char_changes,"links":links})
user_all

,char_changes,dates,links,user


In [ ]:
start = time.time()
for index, row in ipb.iterrows():
    buser = row['rev_user_text']
    bdate = str(row['date'])
    
    # For namespace 3
    
    dates = []
    char_changes = []
    links = []
    flag = 0
    new_url_1 = 'https://en.wikipedia.org/w/index.php?title=Special:Contributions&offset='+bdate+'235959&limit=20&contribs=user&target='+buser+'&namespace=3&tagfilter=&start=&end='
    
    try:
        response1 = requests.get(new_url_1)
        html1 = response1.content
        soup1= BeautifulSoup(html1,'html.parser')
    except:
        exc_count4 = exc_count4 + 1
        print('Connection error at {0}'.format(new_url_1))
        continue
    
    for i in list(soup1.find('div',id='mw-content-text')):
        try:
            if i.get_text() == 'No changes were found matching these criteria.\n':
                flag = 1
        except:
            continue
                    
    if flag == 1:
        exc_count1 = exc_count1+1
        continue
    else:        
        try:
            soup1 = soup1.find('ul',class_='mw-contributions-list')
            lines = soup1.findAll('li')
            if len(lines)<3:
                user_less_than3.append(buser)
                namespace.append('User Talk')
                continue            
        except:
            exc_count2 = exc_count2+1
            continue
        for line in lines:
            date = list(line.children)[0].get_text()
            char_change = list(line.children)[6].get_text()
            li = list(line.children)[2]
            diff = list(li.children)[0]
            if(list(diff.children)[0]=='diff'):
                exc_count3 = exc_count3+1
                continue
            link = list(diff.children)[0].get('href')
            dates.append(date)
            char_changes.append(char_change)
            links.append(link)    
    user = pd.DataFrame({"user": buser,"dates": dates,"char_changes": char_changes,"links":links})
    frames = [user_all,user]
    user_all = pd.concat(frames)

Connection error at https://en.wikipedia.org/w/index.php?title=Special:Contributions&offset=20170704235959&limit=20&contribs=user&target=Ownthelooks&namespace=3&tagfilter=&start=&end=


In [ ]:
for index, row in ipb.iterrows():
    buser = row['rev_user_text']
    bdate = str(row['date'])

# For namespace 1
    
    dates = []
    char_changes = []
    links = []
    flag = 0
    new_url_1 = 'https://en.wikipedia.org/w/index.php?title=Special:Contributions&offset='+bdate+'235959&limit=20&contribs=user&target='+buser+'&namespace=1&tagfilter=&start=&end='
        
    try:
        response1 = requests.get(new_url_1)
        html1 = response1.content
        soup1= BeautifulSoup(html1,'html.parser')
    except:
        exc_count4 = exc_count4 + 1
        print('Connection error at {0}'.format(new_url_1))
        continue
    
    for i in list(soup1.find('div',id='mw-content-text')):
        try:
            if i.get_text() == 'No changes were found matching these criteria.\n':
                flag = 1
        except:
            continue
                    
    if flag == 1:
        exc_count1 = exc_count1+1
        continue
    else:        
        try:
            soup1 = soup1.find('ul',class_='mw-contributions-list')
            lines = soup1.findAll('li')
            if len(lines)<3:
                user_less_than3.append(buser)
                namespace.append('Article Talk')
                continue
        except:
            exc_count2 = exc_count2+1
            continue
        for line in lines:
            date = list(line.children)[0].get_text()
            char_change = list(line.children)[6].get_text()
            li = list(line.children)[2]
            diff = list(li.children)[0]
            if(list(diff.children)[0]=='diff'):
                exc_count3 = exc_count3+1
                continue
            link = list(diff.children)[0].get('href')
            dates.append(date)
            char_changes.append(char_change)
            links.append(link)    
    user = pd.DataFrame({"user": buser,"dates": dates,"char_changes": char_changes,"links":links})
    frames = [user_all,user]
    user_all = pd.concat(frames)
end = time.time()

In [ ]:
print(end - start)

In [ ]:
print(exc_count1,exc_count2,exc_count3,exc_count4)

In [ ]:
user_all

#### Adding required https to link

In [ ]:
user_all['links'] = 'https://en.wikipedia.org'+ user_all['links']
user_all

In [1]:
len(set(user_all['user']))

NameError: name 'user_all' is not defined

4702 users out of 10000 have namespace 1 and 3 diff data available

3338 out of next 10000 have namespace 1 and 3 diff data available

2482 out of next 10000 have namespace 1 and 3 diff data available

2543 out of next 10000 have namespace 1 and 3 diff data available

6272 out of top 11303

In [ ]:
ipb_userlist = set(ipb['rev_user_text'])

In [ ]:
available = set(user_all['user'])

In [ ]:
no_data = ipb_userlist - available
list(no_data)

In [ ]:
len(no_data)

In [ ]:
user_all.to_csv('/home/ec2-user/SageMaker/bucket/wiki_trust/revisions_data/csvs_stored/blocked_more6.csv')

In [ ]:
u_3 = pd.DataFrame({"rev_user_text":user_less_than3,"namespace":namespace})

In [ ]:
u_3

In [ ]:
u_3.to_csv('/home/ec2-user/SageMaker/bucket/wiki_trust/revisions_data/csvs_stored/less_than3_examples3.csv')

### Stage 2 - iterating through all the revision diff links

#### Developing for single user(skip to iterative code)

In [3]:
new_url_1 = list(user_all['links'])[25]
new_url_1

'https://en.wikipedia.org/w/index.php?title=User_talk:Buddha~enwiki&diff=prev&oldid=3081862'

In [4]:
response1 = requests.get(new_url_1)
html1 = response1.content
soup1= BeautifulSoup(html1,'html.parser')
print(soup1.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   User talk:Buddha~enwiki: Difference between revisions - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"User_talk","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":3,"wgPageName":"User_talk:Buddha~enwiki","wgTitle":"Buddha~enwiki","wgCurRevisionId":658232623,"wgRevisionId":3081862,"wgArticleId":579575,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","J

In [5]:
lines = soup1.findAll('td',class_='diff-addedline')
lines

[]

In [24]:
lines[0]

<td class="diff-addedline"><div>''My logic is: Guangdong-Hong Kong is a strong economic/cultural zone. It needs a strong native language of its own. Of course, well, now the Javanese use Bahasa Indonesia, the Venetians use Tuscan and the Punjabis use Urdu. But Cantonese has potentials! Don't let it become another Javanese, Veneto or Punjabi! It should become another Catalan!''</div></td>

In [28]:
type(lines[0])

bs4.element.Tag

In [29]:
lines[0].get_text()

"''My logic is: Guangdong-Hong Kong is a strong economic/cultural zone. It needs a strong native language of its own. Of course, well, now the Javanese use Bahasa Indonesia, the Venetians use Tuscan and the Punjabis use Urdu. But Cantonese has potentials! Don't let it become another Javanese, Veneto or Punjabi! It should become another Catalan!''"

In [30]:
lines[1].get_text()

''

In [31]:
lines[2].get_text()

''

#### Getting this data iteratively

In [2]:
user_all = pd.read_csv('/home/ec2-user/SageMaker/bucket/wiki_trust/revisions_data/csvs_stored/blocked_more3.csv')
del user_all['Unnamed: 0']
user_all

,char_changes,dates,links,user
0,+20,"03:18, 20 January 2018",https://en.wikipedia.org/w/index.php?title=Use...,Holidaysafaritvpm
1,+3,"03:16, 20 January 2018",https://en.wikipedia.org/w/index.php?title=Use...,Holidaysafaritvpm
2,+20,"03:15, 20 January 2018",https://en.wikipedia.org/w/index.php?title=Use...,Holidaysafaritvpm
3,+285,"08:38, 31 March 2017",https://en.wikipedia.org/w/index.php?title=Use...,Speeduptravels
4,+292,"08:37, 31 March 2017",https://en.wikipedia.org/w/index.php?title=Use...,Speeduptravels
5,"-3,156","08:36, 31 March 2017",https://en.wikipedia.org/w/index.php?title=Use...,Speeduptravels
6,"+1,146","08:25, 31 March 2017",https://en.wikipedia.org/w/index.php?title=Use...,Speeduptravels
7,+649,"21:06, 1 October 2017",https://en.wikipedia.org/w/index.php?title=Use...,Rayban101
8,+649,"21:06, 1 October 2017",https://en.wikipedia.org/w/index.php?title=Use...,Rayban101
9,+649,"21:06, 1 October 2017",https://en.wikipedia.org/w/index.php?title=Use...,Rayban101


In [3]:
user_all = user_all.iloc[1850:,]
user_all

,char_changes,dates,links,user
1850,+293,"04:35, 17 November 2017",https://en.wikipedia.org/w/index.php?title=Use...,Pisikyyy
1851,+2,"08:33, 7 July 2017",https://en.wikipedia.org/w/index.php?title=Use...,Iyarkaifoundation
1852,+51,"08:32, 7 July 2017",https://en.wikipedia.org/w/index.php?title=Use...,Iyarkaifoundation
1853,0,"08:30, 7 July 2017",https://en.wikipedia.org/w/index.php?title=Use...,Iyarkaifoundation
1854,+33,"08:30, 7 July 2017",https://en.wikipedia.org/w/index.php?title=Use...,Iyarkaifoundation
1855,-1,"02:00, 26 January 2019",https://en.wikipedia.org/w/index.php?title=Use...,73.60.41.24
1856,"+1,312","01:59, 26 January 2019",https://en.wikipedia.org/w/index.php?title=Use...,73.60.41.24
1857,+820,"02:33, 17 February 2017",https://en.wikipedia.org/w/index.php?title=Use...,73.60.41.24
1858,+824,"02:13, 17 February 2017",https://en.wikipedia.org/w/index.php?title=Use...,73.60.41.24
1859,+397,"17:00, 21 September 2018",https://en.wikipedia.org/w/index.php?title=Use...,Spike Danton


In [4]:
user_all['text'] = ''
user_all

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,char_changes,dates,links,user,text
1850,+293,"04:35, 17 November 2017",https://en.wikipedia.org/w/index.php?title=Use...,Pisikyyy,
1851,+2,"08:33, 7 July 2017",https://en.wikipedia.org/w/index.php?title=Use...,Iyarkaifoundation,
1852,+51,"08:32, 7 July 2017",https://en.wikipedia.org/w/index.php?title=Use...,Iyarkaifoundation,
1853,0,"08:30, 7 July 2017",https://en.wikipedia.org/w/index.php?title=Use...,Iyarkaifoundation,
1854,+33,"08:30, 7 July 2017",https://en.wikipedia.org/w/index.php?title=Use...,Iyarkaifoundation,
1855,-1,"02:00, 26 January 2019",https://en.wikipedia.org/w/index.php?title=Use...,73.60.41.24,
1856,"+1,312","01:59, 26 January 2019",https://en.wikipedia.org/w/index.php?title=Use...,73.60.41.24,
1857,+820,"02:33, 17 February 2017",https://en.wikipedia.org/w/index.php?title=Use...,73.60.41.24,
1858,+824,"02:13, 17 February 2017",https://en.wikipedia.org/w/index.php?title=Use...,73.60.41.24,
1859,+397,"17:00, 21 September 2018",https://en.wikipedia.org/w/index.php?title=Use...,Spike Danton,


In [5]:
start = time.time()
st2_exec_count = 0
for index,row in user_all.iterrows():
    try:
        
        if index==1891:
            print('Skipping adding text at {0}'.format(index))
            continue
        
        new_url_1 = row['links']
    
        response1 = requests.get(new_url_1)
        html1 = response1.content
        soup1= BeautifulSoup(html1,'html.parser')
    
        lines = soup1.findAll('td',class_='diff-addedline')
        txt = ''
        for line in lines:
            txt= txt + line.get_text()
        user_all.at[index,'text'] = txt
        print('Completed adding text at {0}'.format(index))
    except:
        st2_exec_count = st2_exec_count + 1
        print('Exception at {0}'.format(index))
        continue
end = time.time()    

Completed adding text at 1850
Completed adding text at 1851
Completed adding text at 1852
Completed adding text at 1853
Completed adding text at 1854
Completed adding text at 1855
Completed adding text at 1856
Completed adding text at 1857
Completed adding text at 1858
Completed adding text at 1859
Completed adding text at 1860
Completed adding text at 1861
Completed adding text at 1862
Completed adding text at 1863
Completed adding text at 1864
Completed adding text at 1865
Completed adding text at 1866
Completed adding text at 1867
Completed adding text at 1868
Completed adding text at 1869
Completed adding text at 1870
Completed adding text at 1871
Completed adding text at 1872
Completed adding text at 1873
Completed adding text at 1874
Completed adding text at 1875
Completed adding text at 1876
Completed adding text at 1877
Completed adding text at 1878
Completed adding text at 1879
Completed adding text at 1880
Completed adding text at 1881
Completed adding text at 1882
Completed 

Completed adding text at 2124
Completed adding text at 2125
Completed adding text at 2126
Completed adding text at 2127
Completed adding text at 2128
Completed adding text at 2129
Completed adding text at 2130
Completed adding text at 2131
Completed adding text at 2132
Completed adding text at 2133
Completed adding text at 2134
Completed adding text at 2135
Completed adding text at 2136
Completed adding text at 2137
Completed adding text at 2138
Completed adding text at 2139
Completed adding text at 2140
Completed adding text at 2141
Completed adding text at 2142
Completed adding text at 2143
Completed adding text at 2144
Completed adding text at 2145
Completed adding text at 2146
Completed adding text at 2147
Completed adding text at 2148
Completed adding text at 2149
Completed adding text at 2150
Completed adding text at 2151
Completed adding text at 2152
Completed adding text at 2153
Completed adding text at 2154
Completed adding text at 2155
Completed adding text at 2156
Completed 

Completed adding text at 2398
Completed adding text at 2399
Completed adding text at 2400
Completed adding text at 2401
Completed adding text at 2402
Completed adding text at 2403
Completed adding text at 2404
Completed adding text at 2405
Completed adding text at 2406
Completed adding text at 2407
Completed adding text at 2408
Completed adding text at 2409
Completed adding text at 2410
Completed adding text at 2411
Completed adding text at 2412
Completed adding text at 2413
Completed adding text at 2414
Completed adding text at 2415
Completed adding text at 2416
Completed adding text at 2417
Completed adding text at 2418
Completed adding text at 2419
Completed adding text at 2420
Completed adding text at 2421
Completed adding text at 2422
Completed adding text at 2423
Completed adding text at 2424
Completed adding text at 2425
Completed adding text at 2426
Completed adding text at 2427
Completed adding text at 2428
Completed adding text at 2429
Completed adding text at 2430
Completed 

Completed adding text at 2672
Completed adding text at 2673
Completed adding text at 2674
Completed adding text at 2675
Completed adding text at 2676
Completed adding text at 2677
Completed adding text at 2678
Completed adding text at 2679
Completed adding text at 2680
Completed adding text at 2681
Completed adding text at 2682
Completed adding text at 2683
Completed adding text at 2684
Completed adding text at 2685
Completed adding text at 2686
Completed adding text at 2687
Completed adding text at 2688
Completed adding text at 2689
Completed adding text at 2690
Completed adding text at 2691
Completed adding text at 2692
Completed adding text at 2693
Completed adding text at 2694
Completed adding text at 2695
Completed adding text at 2696
Completed adding text at 2697
Completed adding text at 2698
Completed adding text at 2699
Completed adding text at 2700
Completed adding text at 2701
Completed adding text at 2702
Completed adding text at 2703
Completed adding text at 2704
Completed 

Completed adding text at 2946
Completed adding text at 2947
Completed adding text at 2948
Completed adding text at 2949
Completed adding text at 2950
Completed adding text at 2951
Completed adding text at 2952
Completed adding text at 2953
Completed adding text at 2954
Completed adding text at 2955
Completed adding text at 2956
Completed adding text at 2957
Completed adding text at 2958
Completed adding text at 2959
Completed adding text at 2960
Completed adding text at 2961
Completed adding text at 2962
Completed adding text at 2963
Completed adding text at 2964
Completed adding text at 2965
Completed adding text at 2966
Completed adding text at 2967
Completed adding text at 2968
Completed adding text at 2969
Completed adding text at 2970
Completed adding text at 2971
Completed adding text at 2972
Completed adding text at 2973
Completed adding text at 2974
Completed adding text at 2975
Completed adding text at 2976
Completed adding text at 2977
Completed adding text at 2978
Completed 

Completed adding text at 3220
Completed adding text at 3221
Completed adding text at 3222
Completed adding text at 3223
Completed adding text at 3224
Completed adding text at 3225
Completed adding text at 3226
Completed adding text at 3227
Completed adding text at 3228
Completed adding text at 3229
Completed adding text at 3230
Completed adding text at 3231
Completed adding text at 3232
Completed adding text at 3233
Completed adding text at 3234
Completed adding text at 3235
Completed adding text at 3236
Completed adding text at 3237
Completed adding text at 3238
Completed adding text at 3239
Completed adding text at 3240
Completed adding text at 3241
Completed adding text at 3242
Completed adding text at 3243
Completed adding text at 3244
Completed adding text at 3245
Completed adding text at 3246
Completed adding text at 3247
Completed adding text at 3248
Completed adding text at 3249
Completed adding text at 3250
Completed adding text at 3251
Completed adding text at 3252
Completed 

Completed adding text at 3494
Completed adding text at 3495
Completed adding text at 3496
Completed adding text at 3497
Completed adding text at 3498
Completed adding text at 3499
Completed adding text at 3500
Completed adding text at 3501
Completed adding text at 3502
Completed adding text at 3503
Completed adding text at 3504
Completed adding text at 3505
Completed adding text at 3506
Completed adding text at 3507
Completed adding text at 3508
Completed adding text at 3509
Completed adding text at 3510
Completed adding text at 3511
Completed adding text at 3512
Completed adding text at 3513
Completed adding text at 3514
Completed adding text at 3515
Completed adding text at 3516
Completed adding text at 3517
Completed adding text at 3518
Completed adding text at 3519
Completed adding text at 3520
Completed adding text at 3521
Completed adding text at 3522
Completed adding text at 3523
Completed adding text at 3524
Completed adding text at 3525
Completed adding text at 3526
Completed 

In [6]:
print(end - start)

1496.6919195652008


In [7]:
user_all

,char_changes,dates,links,user,text
1850,+293,"04:35, 17 November 2017",https://en.wikipedia.org/w/index.php?title=Use...,Pisikyyy,== The following are twin articles ..pl delete...
1851,+2,"08:33, 7 July 2017",https://en.wikipedia.org/w/index.php?title=Use...,Iyarkaifoundation,We are heading towards with great strength to ...
1852,+51,"08:32, 7 July 2017",https://en.wikipedia.org/w/index.php?title=Use...,Iyarkaifoundation,'''About IYARKAI FOUNDATION (IFN)'''[[File:IYA...
1853,0,"08:30, 7 July 2017",https://en.wikipedia.org/w/index.php?title=Use...,Iyarkaifoundation,[[File:IYARKAI_FOUNDATION.jpg]]
1854,+33,"08:30, 7 July 2017",https://en.wikipedia.org/w/index.php?title=Use...,Iyarkaifoundation,[[File:IYARKAI_FOUNDATION.jpg]]
1855,-1,"02:00, 26 January 2019",https://en.wikipedia.org/w/index.php?title=Use...,73.60.41.24,"Based on my own observations, as well as near-..."
1856,"+1,312","01:59, 26 January 2019",https://en.wikipedia.org/w/index.php?title=Use...,73.60.41.24,== Indigenous People's March incident==Based o...
1857,+820,"02:33, 17 February 2017",https://en.wikipedia.org/w/index.php?title=Use...,73.60.41.24,== Undid revision?? ==Any particular part of m...
1858,+824,"02:13, 17 February 2017",https://en.wikipedia.org/w/index.php?title=Use...,73.60.41.24,== Undid revision 765887968 by [[Special:Contr...
1859,+397,"17:00, 21 September 2018",https://en.wikipedia.org/w/index.php?title=Use...,Spike Danton,"== Bbb23 ==Hello, {{Reply to|Bbb23}} I noticed..."


In [8]:
user_all.to_csv('/home/ec2-user/SageMaker/bucket/wiki_trust/revisions_data/csvs_stored/blocked_more3_2_stage2.csv')

In [9]:
print(st2_exec_count)

1


In [49]:
list(user_all['text'])[0]

"Whatever. It's not over.[[User:$targlass|$targlass]] ([[User talk:$targlass#top|talk]]) 22:47, 19 January 2018 (UTC)"

In [50]:
list(user_all['text'])[1]

'{{unblock|reason=Как Вы определяете фишинговость сайта?}}Какой из его элементов является подозрительным?Вот исходный текст страницы:<html>  <head>    <title>skin.gq</title>    <meta name="description" content="skin.gq">    <meta name="keywords" content="skin.gq">    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">    <script type="text/javascript">        var _gaq = _gaq || [];        _gaq.push([\'_setAccount\', \'UA-23441223-3\']);        _gaq.push([\'_setDomainName\', \'none\']);        _gaq.push([\'_setAllowLinker\', true]);        _gaq.push([\'_trackPageview\']);        (function() { var ga = document.createElement(\'script\'); ga.type = \'text/javascript\'; ga.async = true;            ga.src = (\'https:\' == document.location.protocol ? \'https://ssl\' : \'http://www\') + \'.google-analytics.com/ga.js\';            var s = document.getElementsByTagName(\'script\')[0]; s.parentNode.insertBefore(ga, s);        })();    </script>  </head>  <frameset rows="*">    <

In [51]:
list(user_all['text'])[41]

'== SHCHKSFIGOFSGUICISGUCEGICWGUICUIFWXIFUJAJXFUIAXIUTIAXIUIFXSGUCOSGUICGUOWICWOGUXFUIAXUIAFXUIFX!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!   ==SHCHKSFIGOFSGUICISGUCEGICWGUICUIFWXIFUJAJXFUIAXIUTIAXIUIFXSGUCOSGUICGUOWICWOGUXFUIAXUIAFXUIFX!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'

In [52]:
list(user_all['text'])[42]

'== SHCHKSFIGOFSGUICISGUCEGICWGUICUIFWXIFUJAJXFUIAXIUTIAXIUIFXSGUCOSGUICGUOWICWOGUXFUIAXUIAFXUIFX!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!   ==SHCHKSFIGOFSGUICISGUCEGICWGUICUIFWXIFUJAJXFUIAXIUTIAXIUIFXSGUCOSGUICGUOWICWOGUXFUIAXUIAFXUIFX!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'

### Similarly for Non blocked users

In [2]:
#nonblocked = pd.read_csv("/home/ec2-user/SageMaker/bucket/wiki_trust/revisions_data/csvs_stored/nonblocked_users_1718.csv")
nonblocked = pd.read_csv("/home/ec2-user/SageMaker/bucket/wiki_trust/revisions_data/csvs_stored/ipnb_remaining.csv")
nonblocked.head(20)

,rev_user_text
0,Southern Iowan
1,JezGrove
2,Alexandru M.
3,Minaulos
4,Elfelix
5,KreyszigB
6,ReverseLevity
7,Bensci54
8,192.160.165.63
9,Jfhutson


In [3]:
#ipnb = nonblocked.head(10000)
ipnb = nonblocked.iloc[:10000,]
ipnb

,rev_user_text
0,Southern Iowan
1,JezGrove
2,Alexandru M.
3,Minaulos
4,Elfelix
5,KreyszigB
6,ReverseLevity
7,Bensci54
8,192.160.165.63
9,Jfhutson


In [4]:
buser = []
dates = []
char_changes = []
links = []
exc_count1 = 0
exc_count2 = 0
exc_count3 = 0
exc_count4 = 0
user_all = pd.DataFrame({"user": buser,"dates": dates,"char_changes": char_changes,"links":links})
user_all

,char_changes,dates,links,user


In [5]:
start = time.time()
for index, row in ipnb.iterrows():
    buser = row['rev_user_text']
        
    # For namespace 3
    
    dates = []
    char_changes = []
    links = []
    flag = 0
    new_url_1 = 'https://en.wikipedia.org/w/index.php?limit=20&title=Special%3AContributions&contribs=user&target='+buser+'&namespace=3&tagfilter=&start=&end='
        
    response1 = requests.get(new_url_1)
    html1 = response1.content
    soup1= BeautifulSoup(html1,'html.parser')
    
    try:
        for i in list(soup1.find('div',id='mw-content-text')):
            try:
                if i.get_text() == 'No changes were found matching these criteria.\n':
                    flag = 1
            except:
                continue
    except:
        exc_count4=exc_count4+1
        continue
        
    if flag == 1:
        exc_count1=exc_count1+1
        continue
    else:        
        try:
            soup1 = soup1.find('ul',class_='mw-contributions-list')
            lines = soup1.findAll('li')
            if len(lines)<3:
                #user_less_than3.append(buser)
                continue
        except:
            exc_count2=exc_count2+1
            continue
        for line in lines:
            date = list(line.children)[0].get_text()
            char_change = list(line.children)[6].get_text()
            li = list(line.children)[2]
            diff = list(li.children)[0]
            if(list(diff.children)[0]=='diff'):
                exc_count3=exc_count3+1
                continue
            link = list(diff.children)[0].get('href')
            dates.append(date)
            char_changes.append(char_change)
            links.append(link)    
    user = pd.DataFrame({"user": buser,"dates": dates,"char_changes": char_changes,"links":links})
    frames = [user_all,user]
    user_all = pd.concat(frames)

In [6]:
# For namespace 1
for index, row in ipnb.iterrows():
    buser = row['rev_user_text']

    dates = []
    char_changes = []
    links = []
    flag = 0
    new_url_1 = 'https://en.wikipedia.org/w/index.php?limit=20&title=Special%3AContributions&contribs=user&target='+buser+'&namespace=1&tagfilter=&start=&end='
        
    response1 = requests.get(new_url_1)
    html1 = response1.content
    soup1= BeautifulSoup(html1,'html.parser')
    
    try:
        for i in list(soup1.find('div',id='mw-content-text')):
            try:
                if i.get_text() == 'No changes were found matching these criteria.\n':
                    flag = 1
            except:
                continue
    except:
        exc_count4=exc_count4+1
        continue
                    
    if flag == 1:
        exc_count1=exc_count1+1
        continue
    else:        
        try:
            soup1 = soup1.find('ul',class_='mw-contributions-list')
            lines = soup1.findAll('li')
            if len(lines)<3:
                #user_less_than3.append(buser)
                continue
        except:
            exc_count2=exc_count2+1
            continue
        for line in lines:
            date = list(line.children)[0].get_text()
            char_change = list(line.children)[6].get_text()
            li = list(line.children)[2]
            diff = list(li.children)[0]
            if(list(diff.children)[0]=='diff'):
                exc_count3=exc_count3+1
                continue
            link = list(diff.children)[0].get('href')
            dates.append(date)
            char_changes.append(char_change)
            links.append(link)    
    user = pd.DataFrame({"user": buser,"dates": dates,"char_changes": char_changes,"links":links})
    frames = [user_all,user]
    user_all = pd.concat(frames)
end = time.time() 

In [7]:
print(end - start)

7747.546860933304


In [8]:
print(exc_count1,exc_count2,exc_count3,exc_count4)

4029 0 34881 2


In [9]:
user_all

,char_changes,dates,links,user
0,-28,"01:58, 28 December 2018",/w/index.php?title=User_talk:Southern_Iowan/sa...,Southern Iowan
1,-48,"13:28, 21 December 2018",/w/index.php?title=User_talk:Southern_Iowan/sa...,Southern Iowan
2,-78,"20:36, 15 November 2018",/w/index.php?title=User_talk:Southern_Iowan/sa...,Southern Iowan
3,+36,"20:42, 10 November 2018",/w/index.php?title=User_talk:Southern_Iowan/sa...,Southern Iowan
4,+28,"19:29, 10 November 2018",/w/index.php?title=User_talk:Southern_Iowan/sa...,Southern Iowan
5,+8,"21:06, 9 November 2018",/w/index.php?title=User_talk:Southern_Iowan/sa...,Southern Iowan
6,+12,"21:02, 9 November 2018",/w/index.php?title=User_talk:Southern_Iowan/sa...,Southern Iowan
7,"+13,434","18:46, 9 November 2018",/w/index.php?title=User_talk:Southern_Iowan/sa...,Southern Iowan
8,+2,"10:17, 9 November 2018",/w/index.php?title=User_talk:Southern_Iowan/sa...,Southern Iowan
9,+94,"05:46, 7 November 2018",/w/index.php?title=User_talk:Southern_Iowan/sa...,Southern Iowan


#### Adding required https to link

In [10]:
user_all['links'] = 'https://en.wikipedia.org'+ user_all['links']
user_all

,char_changes,dates,links,user
0,-28,"01:58, 28 December 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
1,-48,"13:28, 21 December 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
2,-78,"20:36, 15 November 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
3,+36,"20:42, 10 November 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
4,+28,"19:29, 10 November 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
5,+8,"21:06, 9 November 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
6,+12,"21:02, 9 November 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
7,"+13,434","18:46, 9 November 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
8,+2,"10:17, 9 November 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
9,+94,"05:46, 7 November 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan


In [11]:
len(set(user_all['user']))

8601

382 users out of 10000 have namespace 1 and 3 diff data available

374 users out of next 30000 have namespace 1 and 3 diff data available

825 users out of next 60000 have namespace 1 and 3 diff data available

3303 users out of next 100000 have namespace 1 and 3 diff data available

2418 users out of next 50000 have namespace 1 and 3 diff data available

1883 users out of next 50000 have namespace 1 and 3 diff data available

3254 users out of next 100000 have namespace 1 and 3 diff data available

538 users out of next 20000 have namespace 1 and 3 diff data available

8601 users out of top 10000

In [12]:
ipnb_userlist = set(ipnb['rev_user_text'])

In [13]:
available = set(user_all['user'])

In [14]:
no_data = ipnb_userlist - available
list(no_data)

['KyleL1Qwerty',
 '24.198.165.63',
 '24.49.101.6',
 '74.93.222.13',
 '75.140.28.143',
 '66.75.109.199',
 'Hickskevin1222',
 '77.179.88.155',
 'Vitor Brto',
 '69.47.62.150',
 '61.102.74.125',
 'Muinax',
 '89.230.230.101',
 'N7jain',
 'Norwegian94',
 '209.33.105.172',
 '124.106.247.203',
 'Angelmx',
 '32.218.67.185',
 'Vasyan174',
 '124.168.116.118',
 'Ak689',
 'Tsao97',
 '50.25.30.233',
 '124.140.68.89',
 'Eric the strong',
 'Benson Iso',
 '99.57.140.55',
 '71.15.168.251',
 '97.115.103.248',
 '207.98.187.174',
 'Tinks737',
 'Usapguy',
 'Flintgamer99',
 'Neelands c',
 'Muhammad Roslee',
 'Nicholascummer',
 '178.59.202.12',
 'Alaskaair',
 'Philip J Fry',
 '2602:306:3A4B:DD60:31E5:9BC9:58B5:D3F4',
 'UpsandDowns1234',
 'Mnikou',
 '216.241.192.33',
 'Patrick9191',
 '2601:83:201:DB83:28FD:320E:1872:421B',
 'Kamkam4545',
 'Saleh aziz',
 '67.81.245.92',
 'SMTWTFS',
 '24.89.216.50',
 'Six 66',
 'Xcalleyfornia',
 '96.54.184.11',
 '68.134.126.38',
 'Mafia Lincoln',
 '24.215.140.98',
 'Bcdfghi',
 '

In [15]:
len(no_data)

1399

In [16]:
user_all.to_csv('/home/ec2-user/SageMaker/bucket/wiki_trust/revisions_data/csvs_stored/nonblocked_top1.csv')

#### Stage 2

In [26]:
user_all = pd.read_csv('/home/ec2-user/SageMaker/bucket/wiki_trust/revisions_data/csvs_stored/nonblocked_top1.csv')
del user_all['Unnamed: 0']
user_all

,char_changes,dates,links,user
0,-28,"01:58, 28 December 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
1,-48,"13:28, 21 December 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
2,-78,"20:36, 15 November 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
3,+36,"20:42, 10 November 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
4,+28,"19:29, 10 November 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
5,+8,"21:06, 9 November 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
6,+12,"21:02, 9 November 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
7,"+13,434","18:46, 9 November 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
8,+2,"10:17, 9 November 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan
9,+94,"05:46, 7 November 2018",https://en.wikipedia.org/w/index.php?title=Use...,Southern Iowan


In [27]:
user_all = user_all.iloc[180000:,]
user_all

,char_changes,dates,links,user
180000,0,"13:07, 23 January 2018",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar
180001,+514,"03:38, 28 November 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar
180002,+298,"02:49, 2 May 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar
180003,"-2,933","02:03, 3 April 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar
180004,+471,"00:57, 18 March 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar
180005,+577,"09:45, 16 March 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar
180006,+564,"18:04, 25 February 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar
180007,+498,"09:42, 2 February 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar
180008,-1,"03:03, 18 November 2016",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar
180009,-1,"23:18, 15 November 2016",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar


In [28]:
user_all['text'] = ''
user_all

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,char_changes,dates,links,user,text
180000,0,"13:07, 23 January 2018",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,
180001,+514,"03:38, 28 November 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,
180002,+298,"02:49, 2 May 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,
180003,"-2,933","02:03, 3 April 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,
180004,+471,"00:57, 18 March 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,
180005,+577,"09:45, 16 March 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,
180006,+564,"18:04, 25 February 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,
180007,+498,"09:42, 2 February 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,
180008,-1,"03:03, 18 November 2016",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,
180009,-1,"23:18, 15 November 2016",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,


In [ ]:
start = time.time()
st2_exec_count = 0
for index,row in user_all.iterrows():
    try:
        new_url_1 = row['links']
    
        response1 = requests.get(new_url_1)
        html1 = response1.content
        soup1= BeautifulSoup(html1,'html.parser')
    
        lines = soup1.findAll('td',class_='diff-addedline')
        txt = ''
        for line in lines:
            txt= txt + line.get_text()
        user_all.at[index,'text'] = txt
    except:
        st2_exec_count = st2_exec_count + 1
        continue
end = time.time()      

In [34]:
print(end - start)

34443.702669620514


In [35]:
print(st2_exec_count)

2


In [36]:
user_all

,char_changes,dates,links,user,text
180000,0,"13:07, 23 January 2018",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,
180001,+514,"03:38, 28 November 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,:I agree with Cjhard. I don't usually like rem...
180002,+298,"02:49, 2 May 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,:That's more about the [[Kobe Yamaguchi-gumi]]...
180003,"-2,933","02:03, 3 April 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,
180004,+471,"00:57, 18 March 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,:::I think you're overstating things. They may...
180005,+577,"09:45, 16 March 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,:That's not in question. The problem is [[WP:O...
180006,+564,"18:04, 25 February 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,":It may be well-known, but it's apparently not..."
180007,+498,"09:42, 2 February 2017",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,:[[Wikipedia:Attribution#Citing_yourself|Citin...
180008,-1,"03:03, 18 November 2016",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,{{sourcecheck|checked=true}}
180009,-1,"23:18, 15 November 2016",https://en.wikipedia.org/w/index.php?title=Tal...,Xanzzibar,{{sourcecheck|checked=true}}


In [ ]:
user_all.to_csv('/home/ec2-user/SageMaker/bucket/wiki_trust/revisions_data/csvs_stored/nonblocked_top1_7_stage2.csv')